In [1]:
%pip install tensorflow tensorflow-hub matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade numpy pandas


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import time
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import cv2

# --- CONFIGURATION ---
INPUT_DIR = "Combined_Study_Data"      # Your 48 images
OUTPUT_DIR = "GenAI_ESRGAN_Results"    # Where the enhanced versions go
MODEL_URL = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

# Create output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"--- Loading GenAI Model (ESRGAN)... ---")
model = hub.load(MODEL_URL)

# --- 1. THE FIXED HELPER FUNCTION (Handles Grayscale) ---
def preprocess_image(image_path):
    """Loads image, forces 3 channels (RGB), and formats for TensorFlow."""
    # Read file
    raw = tf.io.read_file(image_path)
    image = tf.image.decode_image(raw)
    
    # FIX: If grayscale (1 channel), convert to RGB (3 channels)
    if image.shape[-1] == 1:
        image = tf.image.grayscale_to_rgb(image)
    # If 4 channels (PNG alpha), drop the last one
    elif image.shape[-1] == 4:
        image = image[...,:-1]

    image = tf.cast(image, tf.float32)

    # Crop to multiple of 4 (required by model)
    hr_size = (tf.shape(image)[:-1] // 4) * 4
    image = tf.image.crop_to_bounding_box(image, 0, 0, hr_size[0], hr_size[1])
    
    return tf.expand_dims(image, 0)

# --- 2. HELPER TO SAVE RESULT ---
def save_image(image, filename):
    image = tf.squeeze(image)
    image = tf.clip_by_value(image, 0, 255)
    image = tf.cast(image, tf.uint8)
    
    # RGB -> BGR for OpenCV
    image_np = image.numpy()
    image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
    cv2.imwrite(filename, image_bgr)

# --- 3. MAIN LOOP (ALL IMAGES) ---
files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.jpg', '.png'))]

print(f"--- Starting GenAI Enhancement on ALL {len(files)} images ---")
print("This may take 5-10 minutes. Please wait...")

total_start = time.time()

for i, filename in enumerate(files):
    loop_start = time.time()
    
    try:
        # Load & Preprocess
        input_path = os.path.join(INPUT_DIR, filename)
        img_tensor = preprocess_image(input_path)
        
        # Generate (The Slow Part)
        fake_image = model(img_tensor)
        
        # Save
        save_path = os.path.join(OUTPUT_DIR, f"GenAI_{filename}")
        save_image(fake_image, save_path)
        
        # Timing
        elapsed = time.time() - loop_start
        print(f"[{i+1}/{len(files)}] Enhanced {filename} ({elapsed:.1f}s)")
        
    except Exception as e:
        print(f"Error processing {filename}: {e}")

total_time = (time.time() - total_start) / 60
print(f"\nSUCCESS! Processed all images in {total_time:.1f} minutes.")
print(f"Results are in folder: '{OUTPUT_DIR}'")

--- Loading GenAI Model (ESRGAN)... ---
--- Starting GenAI Enhancement on ALL 48 images ---
This may take 5-10 minutes. Please wait...
[1/48] Enhanced headANT0_1100.jpg (6.3s)
[2/48] Enhanced headANT0_1217.jpg (5.5s)
[3/48] Enhanced headANT0_1349.jpg (5.6s)
[4/48] Enhanced headANT0_144.jpg (5.4s)
[5/48] Enhanced headANT0_1494.jpg (5.7s)
[6/48] Enhanced headANT0_154.jpg (5.6s)
[7/48] Enhanced headANT0_1569.jpg (5.5s)
[8/48] Enhanced headANT0_1576.jpg (5.6s)
[9/48] Enhanced headANT0_1604.jpg (5.5s)
[10/48] Enhanced headANT0_1632.jpg (5.6s)
[11/48] Enhanced headANT0_1666.jpg (5.6s)
[12/48] Enhanced headANT0_1714.jpg (5.4s)
[13/48] Enhanced headANT1_111.jpg (5.4s)
[14/48] Enhanced headANT1_1120.jpg (5.4s)
[15/48] Enhanced headANT1_1137.jpg (5.7s)
[16/48] Enhanced headANT1_1138.jpg (5.3s)
[17/48] Enhanced headANT1_1146.jpg (5.7s)
[18/48] Enhanced headANT1_1148.jpg (5.5s)
[19/48] Enhanced headANT1_1158.jpg (5.5s)
[20/48] Enhanced headANT1_1159.jpg (5.7s)
[21/48] Enhanced headANT1_1174.jpg (5

In [ ]:
import shutil
import os

# --- CONFIGURATION ---
# The folder created by the GenAI script
folder_to_zip = "GenAI_ESRGAN_Results"

# The name of the zip file you will download (without .zip)
output_filename = "GenAI_ESRGAN_Output"

print(f"--- Zipping folder: '{folder_to_zip}' ---")

if os.path.exists(folder_to_zip):
    # Create the zip file
    shutil.make_archive(output_filename, 'zip', folder_to_zip)
    
    print(f"Success! Created '{output_filename}.zip'")
    print("Go to the Files pane on the left, click Refresh (⟳), and download the zip file.")
else:
    print(f"Error: Folder '{folder_to_zip}' not found. Did the previous script finish running?")